In [1]:
import networkx as nx
import numpy as np
import math
import random
import math
from scipy.stats import kendalltau
import scipy.io as sio

In [36]:
# Load the .mat file
mat_file = sio.loadmat('E:/M Tech/3. Web & Social Computing/project/code/data/05_Email.mat')

# Get the adjacency matrix

adj_matrix = mat_file['Email']
# adj_matrix = mat_file['polblogs']
# adj_matrix = mat_file['Router5022']
# adj_matrix = mat_file['facebook4039']

# Create a graph from the adjacency matrix
G = nx.from_numpy_array(adj_matrix)
print(nx.number_of_nodes(G), nx.number_of_edges(G))

# gravity radious
R = 3
infect_prob = 0.30


1133 5451


In [3]:
# find average degree of the network
avg_degree = sum(dict(G.degree()).values()) / len(G)
avg_degree

9.62224183583407

In [4]:
# find average distance of the network
avg_distance = nx.average_shortest_path_length(G)
avg_distance

3.606032017315423

In [5]:
########## Degree Centrality ##########
deg_centrality = nx.degree_centrality(G)

list_degree = []
rank_degree = []
# print the degree centrality for each node
for node, centrality in sorted(deg_centrality.items(), key=lambda x: x[1], reverse=True):
    list_degree.append(node)
    rank_degree.append((node, centrality))

In [6]:
########## Betweenness Centrality ##########
btn_centrality = nx.betweenness_centrality(G)

list_btn = []
rank_btn = []
# print the betweenness centrality for each node
for node, centrality in sorted(btn_centrality.items(), key=lambda x: x[1], reverse=True):
    list_btn.append(node)
    rank_btn.append((node, centrality))

In [7]:
########## Closeness Centrality ##########
cc_centrality = nx.closeness_centrality(G)

list_cc = []
rank_cc = []
# print the closeness centrality for each node
for node, centrality in sorted(cc_centrality.items(), key=lambda x: x[1], reverse=True):
    list_cc.append(node)
    rank_cc.append((node, centrality))

In [8]:
# find k-shell of the graph
def k_shell_ranking(G):
    kcores = nx.core_number(G) # calculate core number for each node
    k_max = max(kcores.values()) # find maximum core number

    # initialize dictionary to store k-shell centrality values
    k_shell = dict.fromkeys(G.nodes(), 0)

    # iterate over each k-shell
    for k in range(k_max+1):
        shell_nodes = [node for node in G.nodes() if kcores[node] >= k]

        # if there are no nodes in the k-shell, exit loop
        if len(shell_nodes) == 0:
            break

        # remove nodes in the k-shell and update k-shell centrality values
        G_k = G.subgraph(shell_nodes)
        k_shell_k = nx.core_number(G_k)
        for node in shell_nodes:
            k_shell[node] = k_shell_k[node] + k

    ranking = sorted(k_shell.items(), key=lambda x: x[1], reverse=True)

    return ranking

# call the kshell method 
rank_kshell = k_shell_ranking(G)
list_kshell = [i[0] for i in rank_kshell]

In [9]:
# Find entropy of nodes
def EnRenewRank(G, topk, order):
    # N - 1
    all_degree = nx.number_of_nodes(G) - 1
    # avg degree
    k_ = nx.number_of_edges(G) * 2 / nx.number_of_nodes(G)
    # E<k>
    k_entropy = - k_ * ((k_ / all_degree) * math.log((k_ / all_degree)))

    # node's information pi
    node_information = {}
    for node in nx.nodes(G):
        information = (G.degree(node) / all_degree)
        try:
            node_information[node] = - information * math.log(information)
        except:
            node_information[node] = 0
    # node's entropy Ei
    node_entropy = {}
    for node in nx.nodes(G):
        node_entropy[node] = 0
        for nbr in nx.neighbors(G, node):
            node_entropy[node] += node_information[nbr]

    rank = []
    for i in range(topk):
        # choose the max entropy node
        max_entropy_node, entropy = max(node_entropy.items(), key=lambda x: x[1])
        rank.append((max_entropy_node, entropy))

        cur_nbrs = nx.neighbors(G, max_entropy_node)
        for o in range(order):
            for nbr in cur_nbrs:
                if nbr in node_entropy:
                        node_entropy[nbr] -= (node_information[max_entropy_node] / k_entropy) / (2**o)
            next_nbrs = []
            for node in cur_nbrs:
                nbrs = nx.neighbors(G, node)
                next_nbrs.extend(nbrs)
            cur_nbrs = next_nbrs

        #set the information quantity of selected nodes to 0
        node_information[max_entropy_node] = 0
        # set entropy to 0
        node_entropy.pop(max_entropy_node)
    return rank

# find entropy for all nodes.
rank_entropy = EnRenewRank(G, nx.number_of_nodes(G), 2)
list_entropy = [i[0] for i in rank_entropy]

In [10]:
def get_sir_value(G, node, avg, infect_prob, cover_prob, max_iter):

    time_num_dict_list = []
    time_list = []

    for i in range(avg):
        time, time_num_dict = SIR(G, node.copy(), infect_prob, cover_prob, max_iter)
        time_num_dict_list.append((list(time_num_dict.values())))
        time_list.append(time)
            
    max_time = max(time_list) + 1
    result_matrix = np.zeros((len(time_num_dict_list), max_time))
    for index, (row, time_num_dict) in enumerate(zip(result_matrix, time_num_dict_list)):
        row[:] = time_num_dict[-1]
        row[0:len(time_num_dict)] = time_num_dict
        result_matrix[index] = row
    return np.mean(result_matrix, axis=0)


def SIR(g, infeacted_set, infect_prob, cover_prob, max_iter):

    time = 0
    time_count_dict = {}
    time_count_dict[time] = len(infeacted_set)
    totalNodes = nx.number_of_nodes(g)
    # infeacted_set = infeacted_set
    node_state = {}
    covered_set = set()

    for node in nx.nodes(g):
        if node in infeacted_set:
            node_state[node] = 'i'
        else:
            node_state[node] = 's'

    while len(infeacted_set) != 0 and max_iter != 0:
        ready_to_cover = []
        ready_to_infeact = []
        for node in infeacted_set:
            nbrs = nx.neighbors(g, node)
            for singleNode in list(nbrs):
                if random.uniform(0, 1) <= infect_prob and node_state[singleNode] == 's':
                    node_state[singleNode] = 'i'
                    ready_to_infeact.append(singleNode)
            if random.uniform(0, 1) <= cover_prob:
                ready_to_cover.append(node)
        for node in ready_to_cover:
            node_state[node] = 'r'
            infeacted_set.remove(node)
            covered_set.add(node)
        for node in ready_to_infeact:
            infeacted_set.append(node)
        max_iter -= 1
        time += 1
        time_count_dict[time] = len(covered_set) / totalNodes
    return time, time_count_dict

In [37]:
rank_SIR = []

for i in list(G.nodes()):
    rank_SIR.append([i,get_sir_value(G, [i], 100, infect_prob, 1, 1000)[-1]])
rank_SIR = sorted(rank_SIR, key = lambda x: x[1], reverse=True)
list_SIR = [i[0] for i in rank_SIR]


In [12]:
# Calculate the shortest paths between all pairs of nodes
all_shortest_paths = dict(nx.all_pairs_shortest_path_length(G))

In [13]:
# Calculate the influence of each node
# Using modified Base Gravity model (k-shell + entropy)
influence_scores = {}
for node in G.nodes():
    influence_score = 0
    DKi = dict(rank_entropy)[node] + dict(rank_kshell)[node]
    for other_node in G.nodes():
        distance = all_shortest_paths[node][other_node]
        if node != other_node and R >= distance:
            DKj = dict(rank_entropy)[other_node] + dict(rank_kshell)[other_node]
            strength = DKi * DKj
            influence_score += strength / (distance ** 2)
    influence_scores[node] = influence_score

# Rank the nodes based on their influence scores

sorted_nodes = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)

rank_gravity_entropy = []
list_gravity_entropy = []
for node, score in sorted_nodes:
    rank_gravity_entropy.append((node, score))
    list_gravity_entropy.append(node)

In [14]:
# Calculate the influence of each node
# implemented by base Gravity model (k-shell + degree)
influence_scores = {}
for node in G.nodes():
    influence_score = 0
    DKi = dict(rank_degree)[node] + dict(rank_kshell)[node]
    for other_node in G.nodes():
        distance = all_shortest_paths[node][other_node]
        if node != other_node and R >= distance:
            DKj = dict(rank_degree)[other_node] + dict(rank_kshell)[other_node]
            strength = DKi * DKj
            influence_score += strength / (distance ** 2)
    influence_scores[node] = influence_score

# Rank the nodes based on their influence scores

sorted_nodes = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)

rank_gravity_base = []
list_gravity_base = []
for node, score in sorted_nodes:
    rank_gravity_base.append((node, score))
    list_gravity_base.append(node)

In [15]:
# Calculate the influence of each node
# implemented by GC aglorithm (only k-shell)
influence_scores = {}
for node in G.nodes():
    influence_score = 0
    DKi = dict(rank_kshell)[node]
    for other_node in G.nodes():
        distance = all_shortest_paths[node][other_node]
        if node != other_node and R >= distance:
            DKj = dict(rank_kshell)[other_node]
            strength = DKi * DKj
            influence_score += strength / (distance ** 2)
    influence_scores[node] = influence_score

# Rank the nodes based on their influence scores

sorted_nodes = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)

rank_GC = []
list_GC = []
for node, score in sorted_nodes:
    rank_GC.append((node, score))
    list_GC.append(node)

In [16]:
# Calculate the influence of each node
# implemented by LGM aglorithm (only degree)
influence_scores = {}
for node in G.nodes():
    influence_score = 0
    DKi = dict(rank_degree)[node]
    for other_node in G.nodes():
        distance = all_shortest_paths[node][other_node]
        if node != other_node and R >= distance:
            DKj = dict(rank_degree)[other_node]
            strength = DKi * DKj
            influence_score += strength / (distance ** 2)
    influence_scores[node] = influence_score

# Rank the nodes based on their influence scores

sorted_nodes = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)

rank_LGM = []
list_LGM = []
for node, score in sorted_nodes:
    rank_LGM.append((node, score))
    list_LGM.append(node)

In [17]:
def returnTau(rankList):
    tau, p_value = kendalltau(list_SIR, rankList)
    return tau

In [18]:
def monotonicity(G,rank_list):
  dist = {}
  num = nx.number_of_nodes(G)


  for node,val in rank_list:
    if val in dist:
      dist[val] += 1
    else :
      dist[val] = 1

  sum = 0

  for value in dist.values():
      sum += (value*(value - 1))

  m = sum/(num*(num-1))
  monotonicity = (1 - m)**2

  return monotonicity

In [38]:
# find the Kendall's tau
listOfRanks = {"DC" : list_degree, "Betweenness":list_btn, "Clossness":list_cc, "K-shell":list_kshell, "Entropy":list_entropy, 
               "New Gravity":list_gravity_entropy, "Base Gravity":list_gravity_base, "GC":list_GC, "LGM":list_LGM}
for rankMethod in listOfRanks:
    print(rankMethod + " : " + str(returnTau(listOfRanks[rankMethod])))

DC : 0.2510019055698153
Betweenness : 0.24152395684866781
Clossness : 0.2876786666625083
K-shell : 0.2754312482261983
Entropy : -0.07191264942817312
New Gravity : 0.2708840783560328
Base Gravity : 0.28203680774952516
GC : 0.2861192805616285
LGM : 0.29279345307339405


In [20]:
rankOfMethos = {"DC" : rank_degree, "Betweenness":rank_btn, "Clossness":rank_cc, "K-shell":rank_kshell, "Entropy":rank_entropy, 
               "New Gravity":rank_gravity_entropy, "Base Gravity":rank_gravity_base, "GC":rank_GC, "LGM":rank_LGM}

for rankName in rankOfMethos:
    print(rankName + " : " + str(monotonicity(G, rankOfMethos[rankName])))

DC : 0.8873673021840296
Betweenness : 0.9400088030354118
Clossness : 0.9988027500410694
K-shell : 0.8088125575542995
Entropy : 0.9994075210654945
New Gravity : 0.9999033203985946
Base Gravity : 0.9998970831654469
GC : 0.999893964556168
LGM : 0.9998970831654469
